In [1]:
import sys
import os

In [2]:
class PDBEntry:
    def __init__(self, idx, name, chain):
        self.indexNum = idx
        self.name = name
        self.chain = chain

In [3]:
class single_AA:
    def __init__ (self, seqNum, chain, AA, SS_type):
        self.seqNum = seqNum
        self.chain = chain
        self.AA = AA
        self.SS_type = SS_type

In [4]:
class CSVAggregate:
    def __init__ (self, pdb_name, chain_id):
        self.pdb_name = pdb.name
        self.chain_id = chain_id
        self.PCASSO_AAs = []
        self.PDB_AAs = []
        
    def add_PCASSO_AA(self, seqNum, chain, AA, SS_type ):
        newAA = single_AA(seqNum, chain, AA, SS_type )
        self.PCASSO_AAs.append(newAA)
    
    def add_PDB_AA(self, seqNum, chain, AA, SS_type):
        newAA = single_AA(seqNum, chain, AA, SS_type )
        self.PDB_AAs.append(newAA) 

In [5]:
notebook_path = os.path.abspath("Analysis of CSV files.ipynb")

PDBList = os.path.join(os.path.dirname(notebook_path), "DataSet/Test_Set_300.csv")
PDBs = []
with open(PDBList) as file:
    for item in file:
        item = item.rstrip('\n')
        tmp = item.split(',')
        obj = PDBEntry(tmp[0], tmp[1],tmp[2])
        PDBs.append(obj)
print(len(PDBs))

300


In [6]:
CSVs = []
for pdb in PDBs:
    generic_path =  os.path.join(os.path.dirname(notebook_path),"PDBFiles",pdb.name)
    PCASSO_read_path = os.path.join(generic_path,pdb.name+'.csv')
    PDB_read_path = os.path.join(generic_path,pdb.name+'-pdb.csv')

    PCASSSO_in_file = open(PCASSO_read_path,'r')
    PDB_in_file = open(PDB_read_path,'r')
    lines = PCASSSO_in_file.readlines()
    name = lines[0]
    temp = CSVAggregate(name,pdb.chain)
    
    lines = lines[1:]
    for line in lines:
        parts = line.split(',')
        seqNum = int(parts[0])
        chain = parts[1]
        AA = parts[2]
        SS_type = parts[3][:1]
        temp.add_PCASSO_AA(seqNum, chain, AA, SS_type)
    
    PDB_lines = PDB_in_file.readlines()
    PDB_lines = PDB_lines[1:]
    for line in PDB_lines:
        parts = line.split(',')
        seqNum = int(parts[0])
        chain = parts[1]
        AA = parts[2]
        SS_type = parts[3][:1]
        temp.add_PDB_AA(seqNum, chain, AA, SS_type)    
    CSVs.append(temp)  
    temp = ''

In [7]:
global_true_helix = 0
global_true_sheet = 0
global_true_loop = 0
global_miss_H_as_S = 0
global_miss_H_as_L = 0
global_miss_S_as_H = 0
global_miss_S_as_L = 0
global_miss_L_as_S = 0
global_miss_L_as_H = 0
global_missing_AA = 0
global_PDB_H = 0
global_PDB_S = 0
global_PDB_L = 0
global_PCASSO_H = 0
global_PCASSO_S = 0
global_PCASSO_L = 0
global_total = 0
weirdness = 0

In [8]:
for csv in CSVs:
    local_true_helix = 0
    local_true_sheet = 0
    local_true_loop = 0
    local_miss_H_as_S = 0
    local_miss_H_as_L = 0
    local_miss_S_as_H = 0
    local_miss_S_as_L = 0
    local_miss_L_as_S = 0
    local_miss_L_as_H = 0
    local_missing_AA = 0
    local_mismatch = 0
    PDB_H = 0
    PDB_S = 0
    PDB_L = 0
    PCASSO_H = 0
    PCASSO_S = 0
    PCASSO_L = 0
    
    print(f'PDB {csv.pdb_name} chain {csv.chain_id}')
#     print(csv.PDB_AAs[0].seqNum)
#     print(csv.PDB_AAs[0].chain)
#     print(csv.PDB_AAs[0].AA)
#     print(csv.PDB_AAs[0].SS_type)
#     for x in range(0,5):
    for x in range(len(csv.PCASSO_AAs)):
        PCASSO_temp = csv.PCASSO_AAs[x]

        PDB_temp = single_AA('missing','','','')
                
        for k in range(len(csv.PDB_AAs)):
            if int(csv.PDB_AAs[k].seqNum) == int(PCASSO_temp.seqNum):
                PDB_temp.seqNum = csv.PDB_AAs[k].seqNum
                PDB_temp.chain = csv.PDB_AAs[k].chain
                PDB_temp.AA = csv.PDB_AAs[k].AA
                PDB_temp.SS_type = csv.PDB_AAs[k].SS_type
                break
                
        if PDB_temp.seqNum == 'missing':
            local_missing_AA += 1  
        
        
#         if PDB_temp.seqNum != PCASSO_temp.seqNum:
#             print(f'x:{x} pdb seq {PDB_temp.seqNum} PCASSO seq {PCASSO_temp.seqNum}')
    
#     for x in range(len(csv.PCASSO_AAs)):
#         PCASSO_temp = csv.PCASSO_AAs[x]  
        
#         if x >= len(csv.PDB_AAs):
#             PDB_temp.seqNum = 'missing'
#             PDB_temp.chain = ''
#             PDB_temp.AA = ''
#             PDB_temp.SS_type = 'missing'
#             print('found missing')
#             local_missing_AA += 1            
#         else:
#             PDB_temp = csv.PDB_AAs[x]
                
#         print(f'x:{x} pdb seq {PDB_temp.seqNum} PCASSO seq {PCASSO_temp.seqNum}')
            
        if csv.chain_id == PCASSO_temp.chain:
            pdb_type = PDB_temp.SS_type
            PCASSO_type = PCASSO_temp.SS_type
            
            if PCASSO_type == 'H':
                PCASSO_H += 1
            elif PCASSO_type == 'E':
                PCASSO_S += 1
            elif PCASSO_type == 'C':
                PCASSO_L += 1
            else:
                weirdness += 1
    
            if pdb_type == 'H':
                PDB_H += 1
            elif pdb_type == 'E':
                PDB_S += 1
            elif pdb_type == 'C':
                PDB_L += 1
            else:
                weirdness += 1
                
            if pdb_type == 'H':
                if PCASSO_type == 'H':
                    local_true_helix += 1
                elif PCASSO_type == 'E':
                    local_miss_H_as_S += 1
                elif PCASSO_type == 'C':
                    local_miss_H_as_L += 1
                else:
                    weirdness += 1
                    
            if pdb_type == 'E':
                if PCASSO_type == 'H':
                    local_miss_S_as_H += 1
                elif PCASSO_type == 'E':
                    local_true_sheet += 1
                elif PCASSO_type == 'C':
                    local_miss_S_as_L += 1
                else:
                    weirdness += 1
                    
            if pdb_type == 'C':
                if PCASSO_type == 'H':
                    local_miss_L_as_H += 1
                elif PCASSO_type == 'E':
                    local_miss_L_as_S += 1
                elif PCASSO_type == 'C':
                    local_true_loop += 1
                else:
                    weirdness += 1
            
            if pdb_type != PCASSO_type:
#                 print(f'miss pdb: {pdb_type} PCASSO: {PCASSO_type}')
                local_mismatch += 1
                
            
    
    print(f'\nTrue helix {local_true_helix}, helix as sheet {local_miss_H_as_S}, helix as loop {local_miss_H_as_L}')
    print(f'True sheet {local_true_sheet}, sheet as helix {local_miss_S_as_H},  sheet as loop {local_miss_S_as_L}')
    print(f'True loop {local_true_loop}, loop as helix {local_miss_L_as_H}, loop as sheet {local_miss_L_as_S}\n')
    
    total_miss = local_missing_AA + local_miss_H_as_S + local_miss_H_as_L + local_miss_S_as_H + local_miss_S_as_L + local_miss_L_as_H + local_miss_L_as_S
    
#     print(f'local mismatch direct = {local_mismatch}')
#     print(f'local mismatch calculated = {total_miss}')
    print(f'missing AAs this protein {local_missing_AA}')
    
    
    local_miss_H = local_miss_H_as_S + local_miss_H_as_L
    local_miss_S = local_miss_S_as_H + local_miss_S_as_L
    local_miss_L = local_miss_L_as_H + local_miss_L_as_S
    
    if local_true_helix != 0:
        accuracy_H = local_true_helix / (local_true_helix + local_miss_H)
    else:
        accuracy_H = 0
    
    if local_true_sheet != 0:
        accuracy_S = local_true_sheet / (local_true_sheet + local_miss_S)
    else: 
        accuracy_S = 0
        
    if local_true_loop != 0:
        accuracy_L = local_true_loop / (local_true_loop + local_miss_L)
    else: 
        accuracy_L = 0
        
    print(f'Helix accuracy = {(accuracy_H * 100):.2f}')
    print(f'Sheet accuracy = {(accuracy_S * 100):.2f}')
    print(f'Loop accuracy = {(accuracy_L * 100):.2f}')
    
    local_correct_pred = local_true_helix + local_true_sheet + local_true_loop
    
    local_total_accuracy = local_correct_pred / (local_correct_pred + local_miss_H + local_miss_S + local_miss_L)
    
    
    
    print(f'Total accuracy this protein {local_total_accuracy * 100:.2f}')
    PDB_total = PDB_H + PDB_S + PDB_L
    PCASSO_total = PCASSO_H + PCASSO_S + PCASSO_L
    
#     print(f'PDB_H {PDB_H} PDB_S {PDB_S} PDB_L {PDB_L} TOTAL {PDB_total}')
#     print(f'PCASSO_H {PCASSO_H} PCASSO_S {PCASSO_S} PCASSO_L {PCASSO_L} TOTAL {PCASSO_total}')
    
    if PDB_total != PCASSO_total:
        print(f'Mismatch! PDB_total: {PDB_total} != PCASSO_total: {PCASSO_total}')
    print('\n\n')
    
    global_missing_AA += local_missing_AA
    
    global_true_helix += local_true_helix
    global_true_sheet += local_true_sheet
    global_true_loop += local_true_loop
          
    global_miss_H_as_S += local_miss_H_as_S
    global_miss_H_as_L += local_miss_H_as_L
    global_miss_S_as_H += local_miss_S_as_H
    global_miss_S_as_L += local_miss_S_as_L
    global_miss_L_as_H += local_miss_L_as_H
    global_miss_L_as_S += local_miss_L_as_S
    
    global_PDB_H += PDB_H
    global_PDB_S += PDB_S
    global_PDB_L += PDB_L
    global_PCASSO_H += PCASSO_H
    global_PCASSO_S += PCASSO_S
    global_PCASSO_L += PCASSO_L
    global_total += PCASSO_total
    print(f'running total: {global_total}')

PDB 1C0P chain A

True helix 99, helix as sheet 0, helix as loop 33
True sheet 94, sheet as helix 0,  sheet as loop 5
True loop 120, loop as helix 5, loop as sheet 7

missing AAs this protein 1
Helix accuracy = 75.00
Sheet accuracy = 94.95
Loop accuracy = 90.91
Total accuracy this protein 86.23
Mismatch! PDB_total: 363 != PCASSO_total: 364



running total: 364
PDB 1DG6 chain A

True helix 2, helix as sheet 0, helix as loop 1
True sheet 75, sheet as helix 0,  sheet as loop 3
True loop 65, loop as helix 1, loop as sheet 2

missing AAs this protein 0
Helix accuracy = 66.67
Sheet accuracy = 96.15
Loop accuracy = 95.59
Total accuracy this protein 95.30



running total: 513
PDB 1GCI chain A

True helix 81, helix as sheet 0, helix as loop 3
True sheet 41, sheet as helix 0,  sheet as loop 7
True loop 130, loop as helix 0, loop as sheet 7

missing AAs this protein 0
Helix accuracy = 96.43
Sheet accuracy = 85.42
Loop accuracy = 94.89
Total accuracy this protein 93.68



running total: 782
PDB 


True helix 128, helix as sheet 0, helix as loop 39
True sheet 55, sheet as helix 0,  sheet as loop 5
True loop 140, loop as helix 2, loop as sheet 9

missing AAs this protein 0
Helix accuracy = 76.65
Sheet accuracy = 91.67
Loop accuracy = 92.72
Total accuracy this protein 85.45



running total: 9221
PDB 2ABS chain A

True helix 140, helix as sheet 1, helix as loop 29
True sheet 77, sheet as helix 0,  sheet as loop 2
True loop 84, loop as helix 2, loop as sheet 5

missing AAs this protein 0
Helix accuracy = 82.35
Sheet accuracy = 97.47
Loop accuracy = 92.31
Total accuracy this protein 88.53



running total: 9561
PDB 2B97 chain A

True helix 10, helix as sheet 0, helix as loop 2
True sheet 25, sheet as helix 0,  sheet as loop 3
True loop 30, loop as helix 0, loop as sheet 0

missing AAs this protein 0
Helix accuracy = 83.33
Sheet accuracy = 89.29
Loop accuracy = 100.00
Total accuracy this protein 92.86



running total: 9631
PDB 2BHU chain A

True helix 165, helix as sheet 0, helix as


True helix 229, helix as sheet 0, helix as loop 58
True sheet 276, sheet as helix 0,  sheet as loop 19
True loop 412, loop as helix 2, loop as sheet 18

missing AAs this protein 0
Helix accuracy = 79.79
Sheet accuracy = 93.56
Loop accuracy = 95.37
Total accuracy this protein 90.43



running total: 18107
PDB 3BWZ chain A

True helix 6, helix as sheet 0, helix as loop 3
True sheet 60, sheet as helix 0,  sheet as loop 8
True loop 80, loop as helix 0, loop as sheet 4

missing AAs this protein 0
Helix accuracy = 66.67
Sheet accuracy = 88.24
Loop accuracy = 95.24
Total accuracy this protein 90.68



running total: 18268
PDB 3CBZ chain A

True helix 16, helix as sheet 0, helix as loop 10
True sheet 21, sheet as helix 0,  sheet as loop 9
True loop 43, loop as helix 0, loop as sheet 0

missing AAs this protein 0
Helix accuracy = 61.54
Sheet accuracy = 70.00
Loop accuracy = 100.00
Total accuracy this protein 80.81



running total: 18367
PDB 3CJS chain B

True helix 18, helix as sheet 1, helix


True helix 120, helix as sheet 0, helix as loop 28
True sheet 79, sheet as helix 0,  sheet as loop 6
True loop 114, loop as helix 1, loop as sheet 3

missing AAs this protein 0
Helix accuracy = 81.08
Sheet accuracy = 92.94
Loop accuracy = 96.61
Total accuracy this protein 89.17



running total: 23784
PDB 3QL9 chain A

True helix 38, helix as sheet 0, helix as loop 4
True sheet 13, sheet as helix 0,  sheet as loop 0
True loop 58, loop as helix 4, loop as sheet 7

missing AAs this protein 0
Helix accuracy = 90.48
Sheet accuracy = 100.00
Loop accuracy = 84.06
Total accuracy this protein 87.90



running total: 23908
PDB 3QR7 chain A

True helix 0, helix as sheet 0, helix as loop 0
True sheet 15, sheet as helix 0,  sheet as loop 3
True loop 84, loop as helix 0, loop as sheet 2

missing AAs this protein 0
Helix accuracy = 0.00
Sheet accuracy = 83.33
Loop accuracy = 97.67
Total accuracy this protein 95.19



running total: 24012
PDB 3QY7 chain A

True helix 104, helix as sheet 0, helix as 


True helix 2, helix as sheet 0, helix as loop 2
True sheet 121, sheet as helix 0,  sheet as loop 33
True loop 110, loop as helix 0, loop as sheet 16

missing AAs this protein 0
Helix accuracy = 50.00
Sheet accuracy = 78.57
Loop accuracy = 87.30
Total accuracy this protein 82.04



running total: 33659
PDB 4JCC chain A

True helix 102, helix as sheet 0, helix as loop 46
True sheet 35, sheet as helix 0,  sheet as loop 9
True loop 70, loop as helix 4, loop as sheet 3

missing AAs this protein 0
Helix accuracy = 68.92
Sheet accuracy = 79.55
Loop accuracy = 90.91
Total accuracy this protein 76.95



running total: 33928
PDB 4JIF chain A

True helix 17, helix as sheet 0, helix as loop 13
True sheet 44, sheet as helix 0,  sheet as loop 11
True loop 35, loop as helix 2, loop as sheet 0

missing AAs this protein 0
Helix accuracy = 56.67
Sheet accuracy = 80.00
Loop accuracy = 94.59
Total accuracy this protein 78.69



running total: 34050
PDB 4K5A chain A

True helix 94, helix as sheet 0, helix

Total accuracy this protein 89.27



running total: 43871
PDB 5DBL chain A

True helix 0, helix as sheet 0, helix as loop 0
True sheet 57, sheet as helix 0,  sheet as loop 6
True loop 42, loop as helix 0, loop as sheet 21

missing AAs this protein 0
Helix accuracy = 0.00
Sheet accuracy = 90.48
Loop accuracy = 66.67
Total accuracy this protein 78.57



running total: 43997
PDB 5EMI chain A

True helix 59, helix as sheet 1, helix as loop 18
True sheet 30, sheet as helix 0,  sheet as loop 0
True loop 63, loop as helix 0, loop as sheet 4

missing AAs this protein 0
Helix accuracy = 75.64
Sheet accuracy = 100.00
Loop accuracy = 94.03
Total accuracy this protein 86.86



running total: 44172
PDB 5FAV chain B

True helix 396, helix as sheet 0, helix as loop 85
True sheet 58, sheet as helix 0,  sheet as loop 9
True loop 228, loop as helix 6, loop as sheet 24

missing AAs this protein 0
Helix accuracy = 82.33
Sheet accuracy = 86.57
Loop accuracy = 88.37
Total accuracy this protein 84.62



runn

ZeroDivisionError: division by zero

In [ ]:
print('---=== GLOBAL VALUES ===---')  
print(f'global missing AAs {global_missing_AA}\n')
global_miss_H = global_miss_H_as_S + global_miss_H_as_L
global_miss_S = global_miss_S_as_H + global_miss_S_as_L
global_miss_L = global_miss_L_as_H + global_miss_L_as_S
if global_true_helix != 0:
    global_accuracy_H = global_true_helix / (global_true_helix + global_miss_H)
else:
    global_accuracy_H = 0

if global_true_sheet != 0:
    global_accuracy_S = global_true_sheet / (global_true_sheet + global_miss_S)
else: 
   global_accuracy_S = 0
if global_true_loop != 0:
    global_accuracy_L = global_miss_L / global_true_loop# / (global_true_loop + global_miss_L)
else: 
    global_accuracy_L = 0
    
total_correct = global_true_helix + global_true_sheet + global_true_loop
total_missed = global_miss_H + global_miss_S + global_miss_L
global_total_accuracy = total_correct / (total_correct + total_missed)


print(f'True helix {global_true_helix}, helix as sheet {global_miss_H_as_S}, helix as loop {global_miss_H_as_L}')
print(f'Global Helix accuracy {(global_accuracy_H * 100):.2f}')
print(f'\nTrue sheet {global_true_sheet}, sheet as helix {global_miss_S_as_H},  sheet as loop {global_miss_S_as_L}')
print(f'Global Sheet accuracy {(global_accuracy_S * 100):.2f}')
print(f'\nTrue loop {global_true_loop}, loop as helix {global_miss_L_as_H}, loop as sheet {global_miss_L_as_S}')
print(f'Global Loop accuracy {(global_accuracy_L * 100):.2f}')
print(f'\nGlobal overall accuracy: {global_total_accuracy}')


print('\n')
print(f'PDB_H: {global_PDB_H:9} \t\tPDB_S: {global_PDB_S:8} \t\tPDB_L: {global_PDB_L:8} \t\tTOTAL: {global_PDB_H + global_PDB_S + global_PDB_L}')
print(f'PCASSO_H: {global_PCASSO_H:6} \t\tPCASSO_S {global_PCASSO_S:6} \t\tPCASSO_L: {global_PCASSO_L} \t\tTOTAL: {global_PCASSO_H + global_PCASSO_S + global_PCASSO_L}')
print(f'global total: {global_total}')
